In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### Specifying the result file

Change the value of **resultFile** to be the name/path of the result file generated by the BiLevel_Scheduling_Industry_4.0 program.<br>

In [34]:
resultFile = "./instances60job/resultSPT_VL_LATE2EARLY"
syntheseFile = resultFile+"_Synth.csv"

# Create a synthesis file which will contain the synthesis result
with open(syntheseFile, "w") as file:
    file.write("N;n;m;#inst;#OPT;T_min;T_mean;T_max;d_min;d_mean;d_max\n")



### Extracting data

We only keep columns which will be needed

In [35]:
rawData = pd.read_csv(resultFile+".csv",sep=";", index_col=False)

#print(rawData.columns)

AllData = rawData[['N', 'n', 'm', 'fonction objective (sum wjUj)', 'fonction objective MIP', 'time', 'deviation']]


### Generating dictionnaries

Each dictionnaries are composed of keys with the "N n m" pattern, the value of this key is a list of element<br>
**dico** contains everything related to the data itself<br>
**time_dic** contains the resolution time of the data<br>
**deviation_dic** may or may not contain the deviation of the data whether or not it is non-negative<br>

In [36]:
dico = {}  # { "N n m" : [data] }
time_dic = {}  # { "N n m" : [time] }
deviation_dic = {}  # { "N n m" : [deviation] }

# for each data
for i in range(len(AllData)):
    # Determine its key (category) "N n m"
    data = AllData.iloc[i]
    categorie = str(data['N']) + " " + str(data['n']) + " " + str(data['m'])

    # If this key doesn't exist yet, add the key with an empty list as its value
    if categorie not in dico.keys():
        dico[categorie] = []
        time_dic[categorie] = []
        deviation_dic[categorie] = []

    # append the data, its resolution time into its corresponding list
    dico[categorie].append(data)
    time_dic[categorie].append(data['time'] * 1000)

    # append its deviation if it is non-negative
    if data['deviation'] >= 0:
        deviation_dic[categorie].append(data['deviation'])


### Calculating synthetic result

This bloc of code will append synthetic result into the synthesis file.<br>
These synthetic results are :<br>
min, mean and max resolution time per category<br>
min, mean and max deviation per category<br>
total number of instances<br>
number of instances where the heuristic solution and MIP solution have the same value<br>
<br>
If you change the code, make sure the columns written at the start of the notebook correspond to the values you write in the synthesis file.<br>


In [37]:
for categorie in dico.keys():

    # Resolution time
    min_time = min(time_dic[categorie])
    mean_time = sum(time_dic[categorie]) / len(time_dic[categorie])
    max_time = max(time_dic[categorie])

    #print(categorie, len(deviation_dic[categorie]))

    # Deviation
    min_deviation = min(deviation_dic[categorie])
    mean_deviation = sum(deviation_dic[categorie]) / len(deviation_dic[categorie])
    max_deviation = max(deviation_dic[categorie])

    # total number of instances and number of instances where both resolution are of equal value
    nb_inst = len(dico[categorie])
    nb_opt = 0
    for data in dico[categorie]:
        if data['fonction objective MIP'] == data['fonction objective (sum wjUj)']:
            nb_opt += 1

    # Getting the N, n and m values
    categorie_split = categorie.split(" ")
    N = categorie_split[0]
    n = categorie_split[1]
    m = categorie_split[2]

    # Append synthetic result into the synthesis file
    with open(syntheseFile, "a+") as file:
        # print(N, n, m, nb_inst, nb_opt, min_time, mean_time, max_time, min_deviation, mean_deviation, max_deviation)
        file.write(str(N)+";"+str(n)+";"+str(m)+";"
                   +str(nb_inst)+";"+str(nb_opt)+";"
                   +str(min_time)+";"+str(mean_time)+";"+str(max_time)+";"
                   #+str(len(deviation_dic[categorie]))+";"
                   +str(min_deviation)+";"+str(mean_deviation)+";"+str(max_deviation)+"\n")
    